In [17]:
import pygame as p
import copy
import random
class GameState():
    def __init__(self):
        self.board = [
            ["bR", "bN", "bB", "bQ", "bK", "bB", "bN", "bR"],
            ["bp", "bp", "bp", "bp", "bp", "bp", "bp", "bp"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["--", "--", "--", "--", "--", "--", "--", "--"],
            ["wp", "wp", "wp", "wp", "wp", "wp", "wp", "wp"],
            ["wR", "wN", "wB", "wQ", "wK", "wB", "wN", "wR"]
        ]
        
        self.moveFunctions={'p':self.getPawnMoves,'R':self.getRookMoves,'N':self.getKnightMoves,
                          'B':self.getBishopMoves,'Q':self.getQueenMoves,'K':self.getKingMoves }
        
        self.whiteToMove=True
        self.movelog=[]
        self.whiteKingLocation=(7,4)
        self.blackKingLocation=(0,4)
        self.checkMate=False
        self.staleMate=False
        self.Draw=False
        self.enpassantPossible=()
        self.enPassantPossibleLog=[self.enpassantPossible]
        self.currentCastlingRight=CastleRights(True,True,True,True)
        self.castleRightsLog=[CastleRights(self.currentCastlingRight.wks,self.currentCastlingRight.bks,
         self.currentCastlingRight.wqs,self.currentCastlingRight.bqs)]
        self.board_positions={}
    def makeMove(self,move):
            self.board[move.startRow][move.startCol]="--"
            self.board[move.endRow][move.endCol]=move.pieceMoved
            self.movelog.append(move)
        
            self.whiteToMove=not self.whiteToMove
            if move.pieceMoved=="wK":
                self.whiteKingLocation=(move.endRow,move.endCol)
            if move.pieceMoved=="bK":
                
                
                self.blackKingLocation=(move.endRow,move.endCol)
            if move.isPawnPromotion:
                
                self.board[move.endRow][move.endCol]=move.pieceMoved[0]+"Q"
            ## enpasssant 
            if move.isEnpassantMove:
                self.board[move.startRow][move.endCol]="--"
            
            if move.pieceMoved[1]=="p" and abs(move.startRow-move.endRow)==2:
                self.enpassantPossible=((move.startRow+move.endRow)//2,move.startCol)
            else:
                self.enpassantPossible=()
            ## castle rights
            if move.isCastleMove:
                if move.endCol-move.startCol==2:
                    self.board[move.endRow][move.endCol-1]=self.board[move.endRow][move.endCol+1]
                    self.board[move.endRow][move.endCol+1]="--"
                    
                else:
                    self.board[move.endRow][move.endCol+1]=self.board[move.endRow][move.endCol-2]
                    self.board[move.endRow][move.endCol-2]="--"
            self.enPassantPossibleLog.append(self.enpassantPossible)
            self.updateCastleRights(move)
            self.castleRightsLog.append(CastleRights(self.currentCastlingRight.wks,self.currentCastlingRight.bks,
         self.currentCastlingRight.wqs,self.currentCastlingRight.bqs))
            
        
                
            

    
                
                
            
    def undoMove(self):
        
        if len(self.movelog)!=0:
            
             move=self.movelog.pop()
             self.board[move.startRow][move.startCol]=move.pieceMoved
             
             self.board[move.endRow][move.endCol]=move.pieceCaptured
             self.whiteToMove=not self.whiteToMove
             if move.pieceMoved=="wK":
                self.whiteKingLocation=(move.startRow,move.startCol)
             elif move.pieceMoved=="bK":
                self.blackKingLocation=(move.startRow,move.startCol)
             if move.isEnpassantMove:
                 self.board[move.endRow][move.endCol]="--"
                 self.board[move.startRow][move.endCol]=move.pieceCaptured
             self.enPassantPossibleLog.pop()
             self.enpassantPossible=self.enPassantPossibleLog[-1]
                 
             
             self.castleRightsLog.pop()
             castleRights=copy.deepcopy(self.castleRightsLog[-1])
             self.currentCastlingRight=castleRights
             if move.isCastleMove:
                 
                 if move.endCol-move.startCol==2:
                     self.board[move.endRow][move.endCol+1]=self.board[move.endRow][move.endCol-1]
                     self.board[move.endRow][move.endCol-1]="--"
                 else:
                     self.board[move.endRow][move.endCol-2]=self.board[move.endRow][move.endCol+1]
                     self.board[move.endRow][move.endCol+1]="--"
        self.checkMate=False
        self.staleMate=False
        
    def updateCastleRights(self,move):
        if move.pieceMoved=="wK":
            self.currentCastlingRight.wks=False
            self.currentCastlingRight.wqs=False
        
        elif move.pieceMoved=="bK":
            self.currentCastlingRight.bks=False
            self.currentCastlingRight.bqs=False
        elif move.pieceMoved=="wR":
            if move.startRow==7:
                if move.startCol==0:
                   self.currentCastlingRight.wqs=False
                elif move.startCol==7:
                    self.currentCastlingRight.wks=False
        elif move.pieceMoved=="bR":
            if move.startRow==0:
                if move.startCol==0:
                   self.currentCastlingRight.bqs=False
                elif move.startCol==7:
                    self.currentCastlingRight.bks=False
        if move.pieceCaptured == 'wR':
            if move.endRow == 7:
                if move.endCol == 0:
                    self.currentCastlingRight.wqs = False
                elif move.endCol == 7:
                    self.currentCastlingRight.wks = False
        elif move.pieceCaptured == 'bR':
            if move.endRow == 0:
                if move.endCol == 0:
                    self.currentCastlingRight.bqs = False
                elif move.endCol == 7:
                    self.currentCastlingRight.bks = False
        
            
            
        
                 
    def getValidMoves(self):
        tempEnpassantPossible=self.enpassantPossible
        moves = self.getAllPossibleMoves()  # Corrected method name
        tempCastlingRights=CastleRights(self.currentCastlingRight.wks,self.currentCastlingRight.bks,
                                       self.currentCastlingRight.wqs,self.currentCastlingRight.bqs)
        if self.whiteToMove:
            self.getCastleMoves(self.whiteKingLocation[0],self.whiteKingLocation[1],moves)
        else:
            self.getCastleMoves(self.blackKingLocation[0],self.blackKingLocation[1],moves)
            

        for i in range(len(moves)-1,-1,-1):
            self.makeMove(moves[i])
            self.whiteToMove=not self.whiteToMove
            if self.inCheck():
                moves.remove(moves[i])
            self.whiteToMove=not self.whiteToMove
            self.undoMove()
        if len(moves)==0:
            if self.inCheck():
                
                self.checkMate=True
            else:
                self.staleMate=True
        self.enpassantPossible=tempEnpassantPossible
        self.currentCastlingRight=tempCastlingRights
            
            
        return moves
            
                
        
                
            
            
        
        
    def inCheck(self):
        if self.whiteToMove:
            return self.squareUnderAttack(self.whiteKingLocation[0],self.whiteKingLocation[1])
        else:
            return self.squareUnderAttack(self.blackKingLocation[0],self.blackKingLocation[1])
            
    def squareUnderAttack(self,r,c):
        self.whiteToMove=not self.whiteToMove
        oppMoves=self.getAllPossibleMoves()
        self.whiteToMove=not self.whiteToMove
        for move in oppMoves:
            if move.endRow==r and move.endCol==c:
                
                return True
        
        
        return False
        
        
        
        
    ## MOVES CONSIDERING CHECKS
    def getAllPossibleMoves(self):
        
        moves=[]
        for r in range(8):
            for c in range(8):
                turn=self.board[r][c][0]
                if (turn=="w" and self.whiteToMove) or (turn=="b" and not self.whiteToMove):
                    piece=self.board[r][c][1]
                    self.moveFunctions[piece](r,c,moves)
        return moves
    
    def getPawnMoves(self, r, c, moves):
        
        
        if self.whiteToMove:
            
            
            if self.board[r - 1][c] == "--":
                
                moves.append(Move((r, c), (r - 1, c), self.board))
                if (r == 6 and self.board[r - 2][c] == "--"):
                    
                    moves.append(Move((r, c), (r - 2, c), self.board))
            if c -1>=0:
                if self.board[r-1][c-1][0]=="b":
                    moves.append(Move((r, c), (r - 1, c-1), self.board))
                elif (r-1,c-1)==self.enpassantPossible:
                    moves.append(Move((r, c), (r - 1, c-1), self.board,isEnpassantMove=True))
                    
            if c+1<=7:
                if self.board[r-1][c+1][0]=="b":
                    moves.append(Move((r, c), (r - 1, c+1), self.board))
                elif (r-1,c+1)==self.enpassantPossible:
                    moves.append(Move((r, c), (r - 1, c+1), self.board,isEnpassantMove=True))
        else:
            if self.board[r + 1][c] == "--":
                
                moves.append(Move((r, c), (r + 1, c), self.board))
                if (r == 1 and self.board[r + 2][c] == "--"):
                    
                    moves.append(Move((r, c), (r + 2, c), self.board))
            if c -1>=0:
                if self.board[r+1][c-1][0]=="w":
                    moves.append(Move((r, c), (r + 1, c-1), self.board))
                elif (r+1,c-1)==self.enpassantPossible:
                    moves.append(Move((r, c), (r + 1, c-1), self.board,isEnpassantMove=True))
            if c+1<=7:
                if self.board[r+1][c+1][0]=="w":
                    moves.append(Move((r, c), (r + 1, c+1), self.board))
                elif (r+1,c+1)==self.enpassantPossible:
                    moves.append(Move((r, c), (r + 1, c+1), self.board,isEnpassantMove=True))
                    
            
    
        
            
                
                    
                    
                
            


                
                
                
                
                             
                
                
                             
    def getRookMoves(self,r,c,moves):
        directions=((-1,0),(0,-1),(1,0),(0,1))
        enemyColor="b" if self.whiteToMove else "w"
        for d in directions:
            for i in range(1,8):
                endRow=r+d[0]*i
                endCol=c+d[1]*i
                if 0<=endRow<8 and 0<=endCol<8:
                    
                    endPiece=self.board[endRow][endCol]
                
                    if endPiece=="--":
                        moves.append(Move((r,c),(endRow,endCol),self.board))
                    elif endPiece[0]==enemyColor:
                        moves.append(Move((r,c),(endRow,endCol),self.board))
                        break
                    else:
                        break
                else:
                    break
    def getBishopMoves(self,r,c,moves):
        directions=((-1,-1),(1,-1),(1,1),(-1,1))
        enemyColor="b" if self.whiteToMove else "w"
        for d in directions:
            for i in range(1,8):
                endRow=r+d[0]*i
                endCol=c+d[1]*i
                if 0<=endRow<8 and 0<=endCol<8:
                    
                    endPiece=self.board[endRow][endCol]
                
                    if endPiece=="--":
                        moves.append(Move((r,c),(endRow,endCol),self.board))
                    elif endPiece[0]==enemyColor:
                        moves.append(Move((r,c),(endRow,endCol),self.board))
                        break
                    else:
                        break
                else:
                    break
           
    def getKnightMoves(self,r,c,moves):
        knightMoves=((-1,2),(-1,-2),(1,2),(1,-2),(2,1),(2,-1),(-2,1),(-2,-1))
        allyColor="w" if self.whiteToMove else "b"
        for m in knightMoves:
            endRow=r+m[0]
            endCol=c+m[1]
            if 0<=endRow<8 and 0<=endCol<8:
                endPiece=self.board[endRow][endCol]
                if endPiece[0]!=allyColor:
                    moves.append(Move((r,c),(endRow,endCol),self.board))
            
            
                
            
                
                
                
    def getQueenMoves(self,r,c,moves):
        self.getRookMoves(r,c,moves)
        self.getBishopMoves(r,c,moves)
   
    def getKingMoves(self,r,c,moves):
        kingMoves=((-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1))
        allyColor="w" if self.whiteToMove else "b"
        for i in range(8):
            endRow=r+kingMoves[i][0]
            endCol=c+kingMoves[i][1]
            if 0<=endRow<8 and 0<=endCol<8:
                endPiece=self.board[endRow][endCol]
                if endPiece[0]!=allyColor:
                    moves.append(Move((r,c),(endRow,endCol),self.board))
      
    def getCastleMoves(self,r,c,moves):
        if self.squareUnderAttack(r,c):
            return
        if (self.whiteToMove and self.currentCastlingRight.wks) or (not self.whiteToMove and self.currentCastlingRight.bks):
            self.getKingsideCastleMoves(r,c,moves)
        if (self.whiteToMove and self.currentCastlingRight.wqs) or (not self.whiteToMove and self.currentCastlingRight.bqs):
            self.getQueensideCastleMoves(r,c,moves)
        
    def getKingsideCastleMoves(self,r,c,moves):
        if self.board[r][c+1]=="--" and self.board[r][c+2]=="--":
            if not self.squareUnderAttack(r,c+1) and not self.squareUnderAttack(r,c+2):
                moves.append(Move((r,c),(r,c+2),self.board,isCastleMove=True))
    def getQueensideCastleMoves(self,r,c,moves):
        if self.board[r][c-1] == '--' and self.board[r][c-2] == '--' and self.board[r][c-3] == '--':
            if not self.squareUnderAttack(r, c-1) and not self.squareUnderAttack(r, c-2):
                moves.append(Move((r, c), (r, c-2), self.board, isCastleMove=True))
    def isDrawByFiftyMoves(self):
        if len(self.movelog) >= 50:  # 50 moves by each player
            # Check the last 50 moves for any pawn move or capture
            
            for i in range(1, 51):
                if self.movelog[-i].pieceMoved[1] == 'p' or self.movelog[-i].capture != '--':
                    return False
            return True
        return False
    def isThreeFoldRepetition(self, board):
        
        
    
        # Minimum moves for repetition
        # Check if the last three board positions occurred three times
        current_hash = self.board_hash(board)
        if current_hash in self.board_positions:
            
            self.board_positions[current_hash] += 1
            
            if self.board_positions[current_hash] >= 3:
                
                return True
        else:
            self.board_positions[current_hash] =1
        return False



    

    def board_hash(self,board):
        
    # Convert the board to a hashable string
        return str(board)
        hash_code = ""
        for i in range(8):
            for j in range(8):
                hash_code+=board[i][j]
        return hash_code

   
        
        
        
    # Update the dictionary of board positions
        
    def isDraw(self,board):
        self.Draw = self.isDrawByFiftyMoves() or self.isThreeFoldRepetition(board)
        return self.Draw


            
        
            
                
    
class CastleRights():
    def __init__(self,wks,bks,wqs,bqs):
        self.wks=wks
        self.bks=bks
        self.wqs=wqs
        self.bqs=bqs
        
    
    
    
        
        
        
    
    
        
        



                        
                        
                    
                    
                    
                
                
    

            
       
        
        
       
            
class Move():
    ranksToRows={"1":7,"2":6,"3":5,"4":4,"5":3,"6":2,"7":1,"8":0}
    rowsToRanks={v:k for k,v in ranksToRows.items()}
    filesToCols={"a":0,"b":1,"c":2,"d":3,"e":4,"f":5,"g":6,"h":7}
    colsToFiles={v:k for k,v in filesToCols.items()}
    
    def __init__(self,startSq,endSq,board,isEnpassantMove=False,isCastleMove=False):
        self.startRow=startSq[0]
        self.startCol=startSq[1]
        self.endRow=endSq[0]
        self.endCol=endSq[1]
        self.pieceMoved=board[self.startRow][self.startCol]
        self.pieceCaptured=board[self.endRow][self.endCol]
        self.moveID=self.startRow*1000+self.startCol*100+self.endRow*10+self.endCol
        self.isPawnPromotion=False
        
        if ((self.pieceMoved=="wp" and self.endRow==0) or (self.pieceMoved=="bp" and self.endRow==7)):
             self.isPawnPromotion=True
        
        self.isEnpassantMove=isEnpassantMove
        if self.isEnpassantMove:
            self.pieceCaptured="wp" if self.pieceMoved=="bp" else "bp"
        self.isCastleMove=isCastleMove
        self.moveID=self.startRow*1000+self.startCol*100+self.endRow*10+self.endCol
        
        
        
        
       
    def __eq__(self,other):
        if isinstance(other,Move):
            return self.moveID==other.moveID
        return False
             
    def getChessNotation(self):
        return self.getRankFile(self.startRow,self.startCol)+self.getRankFile(self.endRow,self.endCol)

    def getRankFile(self,r,c):
        return self.colsToFiles[c]+self.rowsToRanks[r]
        
        
        
        
    



In [18]:
pieceScore={"K":0,"Q":9,"R":5,"N":3,"B":3,"p":1,"-":0}
CHECKMATE=1000
STALEMATE=0
DEPTH=3
knightScores=[[1,1,1,1,1,1,1,1],
              [1,2,2,2,2,2,2,1],
             [1,2,4,3,3,3,2,1],
              [1,2,3,5,3,3,2,1],
             [1,2,3,5,3,3,2,1],
              [1,2,3,3,3,3,2,1],
             [1,2,2,2,2,2,2,1],
             [1,1,1,1,1,1,1,1]]
bishopScores=[[4,3,2,1,1,2,3,4],
              [3,4,3,2,2,3,4,3],
             [2,3,4,3,3,4,3,2],
              [1,2,3,4,4,3,3.2,1],
             [1,3.2,3,4,4,3,2,1],
              [2,3,4,3,3,4,3,2],
             [3,4,3,2,2,3,4,3],
             [3,3,2,1,1,2,3,4]]
queenScores=[[1,1,1,3,1,1,1,1],
            [1,2,3,3,3,1,1,1],
            [1,4,2,3,3,2,2,1],
            [1,2,3,3,3,2,2,1],
             [1,2,3,3,3,2,2,1],
            [1,4,3,3,3,2,2,1],
            [1,1,2,3,3,1,1,1],
            [1,1,1,3,1,1,1,1]]
rookScores=[[4,3,4,4,4,4,3,4],
           [4,4,4,4,4,4,4,4],
           [1,1,2,3,3,2,1,1],
           [1,2,3,4,4,3,2,1],
           [1,2,3,4,4,3,2,1],
           [1,1,2,2,2,2,1,1],
           [4,4,4,4,4,4,4,4],
           [4,3,4,4,4,4,3,4]]
whitePawnScores=[[8,8,8,8,8,8,8,8],
                [8,8,8,8,8,8,8,8],
                [5,6,6,6,6,6,6,5],
                [2,3,3,5,5,3,3,2],
                [1,2,3,5,5,3,2,1],
                [1,1,2,3,3,2,1,1],
                [1,1,1,0,0,1,1,1],
                [0,0,0,0,0,0,0,0]]
blackPawnScores=[[0, 0, 0, 0, 0, 0, 0, 0],
[1, 1, 1, 0, 0, 1, 1, 1],
[1, 1, 2, 3, 3, 2, 1, 1],
[1, 2, 3, 5, 5, 3, 2, 1],
[2, 3, 3, 5, 5, 3, 3, 2],
[5, 6, 6, 6, 6, 6, 6, 5],
[8, 8, 8, 8, 8, 8, 8, 8],
[8, 8, 8, 8, 8, 8, 8, 8]]


def adjust_king_scores(gs,board):
    # Check if neither player has a queen
    white_has_queen = any(piece[1] == 'Q' for row in board for piece in row if piece[0] == 'w')
    black_has_queen = any(piece[1] == 'Q' for row in board for piece in row if piece[0] == 'b')
    white_has_rook = any(piece[1] == 'R' for row in board for piece in row if piece[0] == 'w')
    black_has_rook = any(piece[1] == 'R' for row in board for piece in row if piece[0] == 'b')
    white_has_bishop = any(piece[1] == 'B' for row in board for piece in row if piece[0] == 'w')
    black_has_bishop = any(piece[1] == 'B' for row in board for piece in row if piece[0] == 'b')
    
    if not white_has_queen and not black_has_queen and not white_has_rook and not black_has_rook and not white_has_bishop and not black_has_bishop:
        if(gs.whiteToMove):
            
                  
            king_scores = [[1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5],
                         [1,1,1,1,1,1,1,1],
                         [2,2,2,2,2,2,2,2],
                         [2,2,2,3,3,2,2,2],
                         [2,2,2,3,3,2,2,2],
                         [2,2,2,2,3,2,2,2],
                         [1,1,1,2,2,1,1,1],
                         [0,0,1.6,1,1,1.6,0,0],
                         [0,0,0,0,0,0,0,0]]
        else:
            king_scores = [[0,0,0,0,0,0,0,0],
                         [0,0,1.6,1,1,1.6,0,0],
                         [1,1,1,1,1,1,1,1],
                         [2,2,2,2,3,2,2,2],
                         [2,2,2,3,3,2,2,2],
                         [2,2,2,3,3,2,2,2],
                         [2,2,2,2,2,2,2,2],
                         [1,1,1,1,1,1,1,1],
                         [1.5,1.5,1.5,1.5,1.5,1.5,1.5,1.5]]
    else:
        
        king_scores = [
    [0.5, 0, 1.4, 0, 0, 0.5, 1.5, 0.5],
    [0, 0, 0, 0.5, 0.5, 0.5, 0, 0],
    [0, 0, 0, 0.5, 0.5, 0, 0, 0],
    [0, 0, 0, 1, 1, 0, 0, 0],
    [0, 0, 0, 1, 1, 0, 0, 0],
    [0, 0, 0, 0.5, 0.5, 0, 0, 0],
    [0, 0, 0, 0.5, 0, 0, 0, 0],
    [0.5, 0, 1.4, 0, 0, 0.5, 1.5, 0.5]
]
            
            
              
       
      

        # Modify the kingScores matrix to encourage kings to go to the center
       

    return king_scores
piecePositionScores = {
    "N": knightScores,
    "B": bishopScores,
    "Q": queenScores,
    "R": rookScores,
    "wp": whitePawnScores,
    "bp": blackPawnScores,
    
}


def scoreMaterial(gs,board):
    score=0
    for row in board:
        for square in row:
            
            if square[0]=="w":
                score+=pieceScore[square[1]]
            else:
                score-=pieceScore[square[1]]
    return score
            
    

def scoreBoard(gs):
    
        
    
    if gs.checkMate:
        if gs.whiteToMove:
            return -CHECKMATE
        else:
            return CHECKMATE
    elif gs.staleMate:
        return 0
    
    score = 0
    for row in range(len(gs.board)):
        for col in range(len(gs.board[row])):
            
            square = gs.board[row][col]
            if square != "--":
                piecePositionScore = 0
                
                # Scoring based on position
                piece_type = square[1]
                if piece_type == "p":
                    piecePositionScore = piecePositionScores[square][row][col]
                    
                
                    
                    
                elif piece_type!="K":
                    piecePositionScore = piecePositionScores[piece_type][row][col]
                elif piece_type=="K":
                    piecePositionScore=adjust_king_scores(gs,gs.board)[row][col]*2
                    
                if square[0] == "w":
                    score += pieceScore[piece_type] + piecePositionScore * 0.1
                    
                else:
                    score -= pieceScore[piece_type] + piecePositionScore * 0.1
                    
    return score

    



In [19]:
def findRandomMove(validMoves):
    return validMoves[random.randint(0,len(validMoves)-1)]
def findBestMove(gs,validMoves):
    turnMultiplier=1 if gs.whiteToMove else -1
    opponentMinMaxscore=CHECKMATE
    bestPlayerMove=None
    random.shuffle(validMoves)
    for playerMove in validMoves:
        gs.makeMove(playerMove)
        opponentsMoves=gs.getValidMoves()
        if gs.staleMate:
            opponenetMaxScore=STALEMATE
        elif gs.checkMate:
            opponenetMaxScore=-CHECKMATE
        else:
            opponentMaxScore=-CHECKMATE
            for opponentMove in opponentsMoves:
                gs.makeMove(opponentMove)
                gs.getValidMoves()# it is required before checkmate is called 
                if gs.checkMate:
                    
                    score=CHECKMATE
                elif gs.staleMate:
                    
                    score=0
                else:
                    score=-turnMultiplier*scoreMaterial(gs,gs.board)
                if(score>opponentMaxScore):
                    
                    opponentMaxScore=score
                
                gs.undoMove()
        if opponentMinMaxscore > opponentMaxScore:
            
            opponentMinMaxscore=opponentMaxScore
            bestPlayerMove=playerMove
        gs.undoMove()
    return bestPlayerMove
    

    
    

In [20]:
def findMoveMinMax(gs,validMoves,depth,whiteToMove):## helper to make first recursion
    global nextMove,count
    count+=1
    if depth==0:
        return scoreBoard(gs)
    if whiteToMove:
        maxScore=-CHECKMATE
        for move in validMoves:
            gs.makeMove(move)
            nextMoves=gs.getValidMoves()
            score=findMoveMinMax(gs,nextMoves,depth-1,False)
            if score>maxScore:
                maxScore=score
                if depth==DEPTH:
                    nextMove=move
                    print(nextMove.pieceMoved,nextMove.startRow,nextMove.endRow,score)
                    
            gs.undoMove()
        return maxScore
            
            
    else:
        minScore=CHECKMATE
        for move in validMoves:
            gs.makeMove(move)
            nextMoves=gs.getValidMoves()
            score=findMoveMinMax(gs,nextMoves,depth-1,True)
            if score<minScore:
                minScore=score
                if depth==DEPTH:
                    nextMove=move
            gs.undoMove()
        return minScore 
def findMoveNegaMax(gs,validMoves,depth,turnMultiplier):
    global nextMove,count
    count+=1
    if depth==0:
        return turnMultiplier*scoreBoard(gs)
    maxScore=-CHECKMATE
    for move in validMoves:
        gs.makeMove(move)
        nextMoves=gs.getValidMoves()
        score=-findMoveNegaMax(gs,nextMoves,depth-1,-turnMultiplier)
        if score>maxScore:
            maxScore=score
            if depth==DEPTH:
                
                nextMove=move
            
        gs.undoMove()
    return maxScore
def findMoveNegaMaxAlphaBeta(gs,validMoves,depth,alpha,beta,turnMultiplier):
    global nextMove,count
    count+=1
    if depth==0:
        
    
        return turnMultiplier*scoreBoard(gs)
    
    maxScore=-CHECKMATE
    for move in validMoves:
        
        
        gs.makeMove(move)
        nextMoves=gs.getValidMoves()
        score=-findMoveNegaMaxAlphaBeta(gs,nextMoves,depth-1,-beta,-alpha,-turnMultiplier)
        if score>maxScore:
            maxScore=score
            if depth==DEPTH:
                
                nextMove=move
                print(nextMove.pieceMoved,nextMove.startRow,nextMove.startCol,nextMove.endRow,nextMove.endCol,score)
            
        gs.undoMove()
        if maxScore>alpha:
            alpha=maxScore
        if alpha>=beta:
            break
    return maxScore
    
                

In [21]:
def FindBestMove(gs,validMoves):
    global nextMove,count
    count=0
    nextMove=None
    random.shuffle(validMoves)
    findMoveNegaMaxAlphaBeta(gs,validMoves,DEPTH,-CHECKMATE,CHECKMATE,1 if gs.whiteToMove else -1)
    print(count)
    return nextMove

    

In [22]:
p.init()
WIDTH=HEIGHT=512
DIMENSION=8
SQ_SIZE=HEIGHT//DIMENSION
MAX_FPS=15
IMAGES={}


In [23]:
def loadimgs():
    pieces = ["bp", "bR", "bN", "bB", "bK", "bQ", "wp", "wR", "wN", "wB", "wK", "wQ"]
    for piece in pieces:
        IMAGES[piece] = p.transform.scale(p.image.load(piece + ".png"), (SQ_SIZE, SQ_SIZE))


In [24]:
def drawText(screen,text):
    
    
    font=p.font.SysFont("Helvitca",32,True,False)
    textObject=font.render(text,0,p.Color("Gray"))
    textLocation=p.Rect(0,0,WIDTH,HEIGHT).move(WIDTH/2-textObject.get_width()/2,HEIGHT/2-textObject.get_height()/2)
    screen.blit(textObject,textLocation)
    textObject=font.render(text,0,p.Color("Black"))
    screen.blit(textObject,textLocation.move(2,2))
    

In [25]:
def main():
    p.init()
    screen = p.display.set_mode((WIDTH, HEIGHT))
    clock = p.time.Clock()
    screen.fill(p.Color("white"))
    
    gs = GameState()
    
    
    
    animate=False
    validMoves=gs.getValidMoves()
    moveMade=False
    loadimgs()
    sqSelected=()## keep track of last click
    playerClicks=[] ## tarck of couple of clicks
    running = True
    gameOver=False
    playerOne=True # if humna is playing white
    playerTwo=False
    while running:
        humanTurn=(gs.whiteToMove and playerOne) or(not gs.whiteToMove and playerTwo)
        ##humanTurn=True
        for e in p.event.get():
            if e.type == p.QUIT:
                running = False
            elif e.type==p.MOUSEBUTTONDOWN:
                if not gameOver and humanTurn:
                    
                    
                
                
                    location=p.mouse.get_pos()
                    col=location[0]//SQ_SIZE
                    row=location[1]//SQ_SIZE
                    if(sqSelected==(row,col)):
                        
                        sqSelected=()
                        playerClicks=[]
                    else:
                        sqSelected=(row,col)
                        playerClicks.append(sqSelected)
                    if(len(playerClicks))==2:
                        
                        move=Move(playerClicks[0],playerClicks[1],gs.board)
                        print(move.getChessNotation())
                        for i in range(len(validMoves)):
                            if move==validMoves[i]:
                                
                            
                        
                                animate=True
                        
                                gs.makeMove(validMoves[i])
                                moveMade=True
                                sqSelected=()
                                playerClicks=[]
                        
                        if not moveMade:
                            playerClicks=[sqSelected]
                        
            elif e.type == p.KEYDOWN:
                if e.key==p.K_z:
                    gs.undoMove()
                    moveMade=True
                    animate=False
                    gameOver=False
                if e.key==p.K_r:
                    gs=GameState()
                    validMoves=gs.getValidMoves()
                    sqSelected=()
                    playerClicks=[]
                    moveMade=False
                    animate=False
                    gameOver=False
        ##AI MOVE FINDER
        if not gameOver and not humanTurn:
            AIMove=FindBestMove(gs,validMoves)
            if AIMove is None:
                AIMove=findRandomMove(validMoves)
                
            gs.makeMove(AIMove)
            moveMade=True
            #nimate=True
            
        if moveMade:
            if animate:
                
                
                animateMove(gs.movelog[-1],screen,gs.board,clock)
            validMoves=gs.getValidMoves()
            moveMade=False
            animate=False
        drawGameState(screen, gs,validMoves,sqSelected)
        if gs.checkMate:
            
            
            gameOver=True
            if gs.whiteToMove:
                
                
                
                drawText(screen,"Black wins by checkmate")
            else:
                
                
                drawText(screen,"white wins by checkmate")
        elif gs.staleMate:
            
            
            
            
            
            gameOver=True
            
            
            drawText(screen,"draw by stalemate")
       ## elif gs.isDraw(gs.board)==True:
           ## gameOver=True
            ##drawText(screen,"draw by three fold or 50-moves")
            
            
                
                    
            
                    
                    
                    
                    
                
                
          # Draw the game state
            
        clock.tick(MAX_FPS)
        p.display.flip()




In [26]:
def highlightSquares(screen,gs,validMoves,sqSelected):
    if sqSelected!=():
        r,c=sqSelected
        if gs.board[r][c][0]==("w" if gs.whiteToMove else "b"):
            s=p.Surface((SQ_SIZE,SQ_SIZE))
            s.set_alpha(100)#transparency value
            s.fill(p.Color("blue"))
            screen.blit(s,(c*SQ_SIZE,r*SQ_SIZE))
            s.fill(p.Color("yellow"))
            for move in validMoves:
                if move.startRow==r and move.startCol==c:
                    screen.blit(s,(SQ_SIZE*move.endCol,SQ_SIZE*move.endRow))


In [27]:
## responsible for all grapjics from board to pieces to events and inputs
def drawGameState(screen,gs,validMoves,sqSelected):
    drawBoard(screen) ##amke squares
    highlightSquares(screen,gs,validMoves,sqSelected)
    drawPieces(screen,gs.board) ## put pieces over sqf
    
    

In [28]:
def drawBoard(screen):
    global colors
    colors=[p.Color("white"),p.Color("gray")]
    for r in range(DIMENSION):
        for c in range(DIMENSION):
            color=colors[((r+c)%2)]
            p.draw.rect(screen,color,p.Rect(c*SQ_SIZE,r*SQ_SIZE,SQ_SIZE,SQ_SIZE))
            
                
            



In [29]:
def drawPieces(screen,board):
    for row in range(DIMENSION):
        for c in range(DIMENSION):
            piece=board[row][c]
            if piece !="--":
                screen.blit(IMAGES[piece],p.Rect(c*SQ_SIZE,row*SQ_SIZE,SQ_SIZE,SQ_SIZE))



In [30]:
def animateMove(move,screen,board,clock):
    global colors
    
    dR=move.endRow-move.startRow
    dC=move.endCol-move.startCol
    framesPerSquare=8
    frameCount=framesPerSquare*(abs(dR)+abs(dC))
    for frame in range(frameCount+1):
        r,c=(move.startRow+dR*(frame/frameCount),move.startCol+dC*(frame/frameCount))
        drawBoard(screen)
        drawPieces(screen,board)
        color=colors[(move.endRow+move.endCol)%2]
        endSquare=p.Rect(move.endCol*SQ_SIZE,move.endRow*SQ_SIZE,SQ_SIZE,SQ_SIZE)
        p.draw.rect(screen,color,endSquare)
        if move.pieceCaptured!="--":
                 
                 screen.blit(IMAGES[move.pieceCaptured],endSquare)
        screen.blit(IMAGES[move.pieceMoved],p.Rect(c*SQ_SIZE,r*SQ_SIZE,SQ_SIZE,SQ_SIZE))
        p.display.flip()
        clock.tick(60)
    
    
    
    

In [31]:
main()

FileNotFoundError: No file 'bB.png' found in working directory 'C:\Users\rasha\OneDrive\Desktop\Chess AI'.